In [1]:
import glob
import os
import pathlib

import numpy as np
import pandas as pd

from util.image import unnormalize

import torch
import torchvision
from torch.utils.data import Dataset, DataLoader
from torch import nn
from torch import functional as F

from torchvision import transforms
from torchinfo import torchinfo
from tqdm import tqdm
import matplotlib.pyplot as plt

import albumentations as A
import torch.nn.functional as F

from PIL import Image

import torchmetrics
from torchvision.utils import save_image, make_grid
import cv2
from util.io import load_ckpt

from util.loss import  InpaintingLoss
import os, glob

import efficientunet
import random

device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device("cpu")

c:\ProgramData\Anaconda3\envs\cvproject\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset_path = 'Datasets/Mask detection/archive/Face Mask Dataset/'
train_dir = dataset_path+'Train/'
val_dir = dataset_path+'Validation/'
test_dir = dataset_path+'Test/'

In [3]:
sizes = (64, 64)

rescale_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize(sizes, antialias= False)
])

In [4]:
torch.hstack([torch.ones(5), torch.zeros(4)])

tensor([1., 1., 1., 1., 1., 0., 0., 0., 0.])

In [5]:
def get_files(path):
    mask_files = glob.glob(path + 'WithMask/' +'*.png')
    nomask_files = glob.glob(path + 'WithoutMask/' +'*.png')

    mask_images = [rescale_transform(Image.open(x)) for x in tqdm(mask_files)]
    unmasked_images = [rescale_transform(Image.open(x)) for x in tqdm(nomask_files)]

    mask_labels = torch.ones(len(mask_images))
    unmask_labels = torch.zeros(len(unmasked_images))

    mask_images = torch.stack(mask_images)
    unmasked_images = torch.stack(unmasked_images)
    images = torch.vstack([mask_images, unmasked_images])
    labels = torch.hstack([mask_labels, unmask_labels])

    return images, labels

In [6]:
train_images, train_labels = get_files(train_dir)

100%|██████████| 5000/5000 [00:14<00:00, 341.31it/s]


In [7]:
val_images, val_labels = get_files(val_dir)

100%|██████████| 400/400 [00:01<00:00, 390.79it/s]


In [8]:
class MaskDataset(Dataset):
    def __init__(self, images, labels):
        self.images = images
        self.labels = labels

    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        return self.images[idx], self.labels[idx]

In [9]:
train_dataset = MaskDataset(train_images, train_labels)
val_dataset = MaskDataset(val_images, val_labels)

In [10]:
BATCH_SIZE = 250

train_dataloader = DataLoader(train_dataset, BATCH_SIZE, shuffle = True)
val_dataloader   = DataLoader(val_dataset, BATCH_SIZE, shuffle = False)

In [15]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1 = nn.Conv2d(3, 5, 7)        # out = 58
        self.conv2 = nn.Conv2d(5, 5, 3)       # in = 29, out = 13

        self.pool = nn.MaxPool2d(2)

        self.fc1 = nn.Linear(5 * 13 * 13, 64)
        self.o_n = nn.Linear(64, 1)


        self.flatten = nn.Flatten()
        self.activation = nn.ReLU()

    def forward(self, inpt):
        out = self.activation(self.conv1(inpt))
        out = self.pool(out)
        
        out = self.activation(self.conv2(out))
        out = self.pool(out)

        out = self.flatten(out)

        out = self.activation(self.fc1(out))
        out = self.o_n(out)

        return out

In [16]:
class EarlyStopping:

    def __init__(self, patience=5, verbose=False, delta=0, path='checkpoint.pt', trace_func=print):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path
        self.trace_func = trace_func
        
    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)

        elif score < self.best_score + self.delta:
            self.counter += 1
            self.trace_func(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True

        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        if self.verbose:
            self.trace_func(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

In [17]:
torchinfo.summary(CNN(), (5, 3, 64, 64))

Layer (type:depth-idx)                   Output Shape              Param #
CNN                                      [5, 1]                    --
├─Conv2d: 1-1                            [5, 5, 58, 58]            740
├─ReLU: 1-2                              [5, 5, 58, 58]            --
├─MaxPool2d: 1-3                         [5, 5, 29, 29]            --
├─Conv2d: 1-4                            [5, 5, 27, 27]            230
├─ReLU: 1-5                              [5, 5, 27, 27]            --
├─MaxPool2d: 1-6                         [5, 5, 13, 13]            --
├─Flatten: 1-7                           [5, 845]                  --
├─Linear: 1-8                            [5, 64]                   54,144
├─ReLU: 1-9                              [5, 64]                   --
├─Linear: 1-10                           [5, 1]                    65
Total params: 55,179
Trainable params: 55,179
Non-trainable params: 0
Total mult-adds (M): 13.56
Input size (MB): 0.25
Forward/backward pass size (MB

In [18]:
EPOCHS = 25

train_acc = torchmetrics.classification.BinaryAccuracy().to(device)
val_acc = torchmetrics.classification.BinaryAccuracy().to(device)

model = CNN().to(device)
optim = torch.optim.Adam(model.parameters(), lr = 1e-3)
criterion = torch.nn.BCELoss()
early_stopping = EarlyStopping(patience=3, verbose=True, path = 'mask_model.pth')


for epoch_num in range(EPOCHS):
    train_loss = 0
    i = 0

    bar = tqdm(train_dataloader)
    for img, label in bar:
        i+=1
        optim.zero_grad()

        img = img.to(device)
        label = label.to(device).unsqueeze(1)
        predictions = F.sigmoid(model(img))

        batch_loss = criterion(predictions, label)

        train_acc(predictions, label)
        batch_loss.backward()
        optim.step()

        train_loss+= batch_loss.item()
        bar.set_description_str("Training loss: {:.4f}, accuracy = {:.4f}".format(train_loss/i, train_acc.compute()))

    train_loss/=i

    
    with torch.no_grad():
        val_loss = 0
        i = 0
        bar = tqdm(val_dataloader)
        for img, label in bar:
            i+=1
            optim.zero_grad()

            img = img.to(device)
            label = label.to(device).unsqueeze(1)
            predictions = F.sigmoid(model(img))

            batch_loss = criterion(predictions, label)

            
            val_acc(predictions, label)
            val_loss+= batch_loss.item()
            bar.set_description_str("Validation loss: {:.4f}, accuracy = {:.4f}".format(val_loss/i, val_acc.compute()))

        val_loss/=i


    print("Epoch [{}/{}], Train Loss: {:.4f}, Train Accuracy: {:.4f}".format(epoch_num+1, EPOCHS, train_loss, train_acc.compute()))
    print("Epoch [{}/{}], Val Loss: {:.4f}, Val Accuracy: {:.4f}".format(epoch_num+1, EPOCHS, val_loss, val_acc.compute()))
    early_stopping(val_loss, model)

    train_acc.reset()
    val_acc.reset()

    if early_stopping.early_stop:
        print("Early stopping")
        print('-'*60)
        break

Validation loss: 0.2635, accuracy = 0.8950: 100%|██████████| 4/4 [00:00<00:00, 121.58it/s]


Epoch [1/25], Train Loss: 0.4465, Train Accuracy: 0.7924
Epoch [1/25], Val Loss: 0.2635, Val Accuracy: 0.8950
Validation loss decreased (inf --> 0.263547).  Saving model ...


Validation loss: 0.1868, accuracy = 0.9350: 100%|██████████| 4/4 [00:00<00:00, 139.06it/s]


Epoch [2/25], Train Loss: 0.2075, Train Accuracy: 0.9213
Epoch [2/25], Val Loss: 0.1868, Val Accuracy: 0.9350
Validation loss decreased (0.263547 --> 0.186780).  Saving model ...


Validation loss: 0.1797, accuracy = 0.9425: 100%|██████████| 4/4 [00:00<00:00, 132.98it/s]


...

Epoch [23/25], Train Loss: 0.0225, Train Accuracy: 0.9925
Epoch [23/25], Val Loss: 0.0199, Val Accuracy: 0.9937
Validation loss decreased (0.021504 --> 0.019858).  Saving model ...


Validation loss: 0.0183, accuracy = 0.9950: 100%|██████████| 4/4 [00:00<00:00, 132.73it/s]


Epoch [24/25], Train Loss: 0.0191, Train Accuracy: 0.9933
Epoch [24/25], Val Loss: 0.0183, Val Accuracy: 0.9950
Validation loss decreased (0.019858 --> 0.018292).  Saving model ...


Validation loss: 0.0162, accuracy = 0.9962: 100%|██████████| 4/4 [00:00<00:00, 128.75it/s]

Epoch [25/25], Train Loss: 0.0193, Train Accuracy: 0.9934
Epoch [25/25], Val Loss: 0.0162, Val Accuracy: 0.9962
Validation loss decreased (0.018292 --> 0.016159).  Saving model ...
